# Setup

In [79]:
import sys
import time

sys.path.append('../utils')
from pyutils import *

In [80]:
sample="2333133121414131402"

In [81]:
with open('input.txt', 'r', encoding='utf-8') as f:
    puzzle = f.read()

In [82]:
def read_diskmap(dm: str) -> list[int]:
    processed: list[str] = []
    mode: int = 1 # 1 == file, -1 == free space
    files: int = -1
    for digit in dm:
        if not digit.isnumeric():
            continue
        digit = int(digit)
        if mode == 1:
            files += 1
            processed.extend([files] * digit)
        elif mode == -1:
            processed.extend([-1] * digit)
        mode *= -1
    return processed

In [83]:
def block_report(d: list[int]):
    groups = {'used': {}, 'free': []}
    freeblock = 0
    for n, fid in enumerate(d):
        if fid == -1:
            freeblock += 1
        else:
            if freeblock > 0:
                groups['free'].append((n - freeblock, freeblock))
                freeblock = 0
            if fid not in groups['used']:
                groups['used'][fid] = (n, [])
            groups['used'][fid][1].append(fid)
    return groups

In [84]:
block_report(disk)

{'used': {0: (0, [0, 0]),
  1: (5, [1, 1, 1]),
  2: (11, [2]),
  3: (15, [3, 3, 3]),
  4: (19, [4, 4]),
  5: (22, [5, 5, 5, 5]),
  6: (27, [6, 6, 6, 6]),
  7: (32, [7, 7, 7]),
  8: (36, [8, 8, 8, 8]),
  9: (40, [9, 9])},
 'free': [(2, 3),
  (8, 3),
  (12, 3),
  (18, 1),
  (21, 1),
  (26, 1),
  (31, 1),
  (35, 1)]}

In [94]:
def optimize_disk(d: list[int], defrag: bool=True) -> list[int]:
    d = d.copy()
    report = block_report(d)
    report['used'] = {k:report['used'][k] for k in sorted(report['used'].keys(), reverse=True)}
    for ubid, ublock in report['used'].items():
        time.sleep(1)
        print(''.join(map(str, d)).replace('-1', '.'))
        for fbid, fblock in enumerate(report['free']):
            fblock = report['free'].pop(fbid)
            if fblock[1] >= len(ublock[1]):
                # Replace free space with file
                d[fblock[0]:fblock[0] + len(ublock[1])] = ublock[1]
                # Replace previous file space with free space
                d[ublock[0]:ublock[0] + len(ublock[1])] = ['-1'] * len(ublock[1])
                # Add free block back into same list position, adjust index and shrink size
                report['free'].insert(fbid, (fblock[0] + len(ublock[1]), fblock[1] - len(ublock[1])))
                break
    # for n in range(len(d)):
    #     n = abs(n - len(d)) - 1
    #     if d[n] != -1:
    #         leftmost_free = d.index(-1)
    #         if leftmost_free > n:
    #             # All sorted, no more work to do
    #             break
    #         d[leftmost_free], d[n] = d[n], d[leftmost_free]
    return d

In [86]:
def calc_disk_checksum(d: list[int], show: bool=False) -> str:
    chsum: int = 0
    for n, digit in enumerate(d):
        if show:
            time.sleep(0.005)
            clear_output(wait=True)
            print(f'{repr(n):^8}{repr(digit):^8}{chsum}')
        if digit == -1:
            break
        chsum += n * digit
    return chsum

# Solve

In [87]:
disk: list[int] = read_diskmap(sample)

In [95]:
ta = time.perf_counter()
optimized = optimize_disk(disk)
tb = time.perf_counter()
print(f'Optimized disk of length {len(optimized)} in {tb - ta:.05f}s')

00...111...2...333.44.5555.6666.777.888899
0099.111...2...333.44.5555.6666.777.8888..
0099.111...2...333.44.5555.6666.777.8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
0099.1117772...333.44.5555.6666.....8888..
Optimized disk of length 42 in 10.00614s


In [69]:
calc_disk_checksum(optimized, show=False)

45

In [93]:
print(''.join(map(str, disk)).replace('-1', '.'))

00...111...2...333.44.5555.6666.777.888899
